In [14]:
import pandas as pd
from extract_features import extract_features
from read_conllu_to_csv import read_conllu_write_csv
from read22 import split_gold_column

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
read_conllu_write_csv('../UP-1.0/input/en_ewt-up-dev.conllu', '../UP-1.0/output/read_conllu_dev.csv')
extract_features('../UP-1.0/output/read_conllu_dev.csv', '../UP-1.0/features/extract_features_out_dev.csv')

File read_conllu_dev.csv produced!
exctacted voice


In [12]:
df = pd.read_csv( '../UP-1.0/features/extract_features_out_dev.csv', sep='\t')
preds, args = split_gold_column(df.Label.tolist())
ident_args = [1 if a != '_' else 0 for a in args]
df['Identify args'] = ident_args
df.head(10)

C:\Users\snipercapt\AppData\Local\Temp\ipykernel_12740\2440933527.py:1: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv( '../UP-1.0/features/extract_features_out_dev.csv', sep='\t')


,Token,PoS,Lemma,Dependency,Head,Head_POS,Morphological Feature,Path to head texts,Path to head POS,Label,Entity_type,E_DEP,voice,Gold,Identify args
0,1,NUM,1,dep,comes,VERB,NumType=Card,"['1', 'comes']","['NUM', 'VERB']",11,NaN,NaN,NaN,10,1
1,From,ADP,from,prep,comes,VERB,NaN,"['From', 'comes']","['ADP', 'VERB']",_,NaN,NaN,NaN,_,0
2,the,DET,the,det,AP,PROPN,Definite=Def|PronType=Art,"['the', 'AP', 'From', 'comes']","['DET', 'PROPN', 'ADP', 'VERB']",_,NaN,NaN,NaN,_,0
3,AP,PROPN,AP,pobj,From,ADP,Number=Sing,"['AP', 'From', 'comes']","['PROPN', 'ADP', 'VERB']",ARG2,NaN,NaN,NaN,_,1
4,comes,VERB,come,ccomp,nominated,VERB,Number=Sing|Person=3|Tense=Pres|VerbForm=Fin,['comes'],['VERB'],V,NaN,NaN,NaN,come.03,0
5,this,DET,this,det,story,NOUN,Number=Sing|PronType=Dem,"['this', 'story', 'comes']","['DET', 'NOUN', 'VERB']",_,NaN,NaN,NaN,_,0
6,story,NOUN,story,dobj,comes,VERB,Number=Sing,"['story', 'comes']","['NOUN', 'VERB']",ARG1,NaN,NaN,NaN,_,1
7,:,PUNCT,:,punct,nominated,VERB,NaN,[':'],['PUNCT'],_,NaN,NaN,NaN,_,0
8,President,PROPN,President,compound,Bush,PROPN,Number=Sing,"['President', 'Bush']","['PROPN', 'PROPN']",ARG0,NaN,NaN,NaN,_,1
9,Bush,PROPN,Bush,nsubj,nominated,VERB,Number=Sing,['Bush'],['PROPN'],_,NaN,NaN,NaN,_,0


In [23]:
vec = DictVectorizer()

df = df.dropna(axis='columns')
X = df.drop(['Identify args', 'Label'], axis=1)
y = df['Identify args']

X_vec = vec.fit_transform(X.to_dict('records'))

In [25]:
model = LogisticRegression(max_iter=1000).fit(X_vec, y)

In [26]:
f1_score(model.predict(X_vec), y)

0.993345052737318